In [2]:
import sys, os
sys.path.insert(0, os.getcwd())
sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0,os.path.join(os.path.dirname(os.getcwd()), "testing"))
from testing.isi_corr import *

def dnn_vs_um(mt_path:str):
    mt = pd.read_csv(mt_path)
    sessions = set(mt["RecSes1"].unique())
    dnn_auc, um_auc, numbers_d, numbers_u = [], [], [], []
    for r1 in tqdm(sessions):
        for r2 in tqdm(sessions):
            if r1>=r2:
                continue
            dnn, um, n_dnn, n_um = auc_one_pair(mt, r1, r2, mt_path=mt_path, dist_thresh=20)
            if dnn is not None and um is not None:
                dnn_auc.append(dnn)
                numbers_d.append(n_dnn)
                um_auc.append(um)
                numbers_u.append(n_um)
    fig = plt.figure()
    ax1 = fig.add_subplot(121)
    ax1.scatter(um_auc, dnn_auc)
    ax1.set(title="AUC", xlabel="UnitMatch", ylabel="DNN")
    ax2 = fig.add_subplot(122)
    ax2.scatter(numbers_u, numbers_d)
    ax2.set(title="Number of matches", xlabel="UnitMatch", ylabel="DNN")
    fig.tight_layout()
    plt.show()

In [4]:
test_data_root = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "R_DATA_UnitMatch")
mt_path = os.path.join(test_data_root, "AL036", "19011116882", "3", "new_matchtable.csv")

dnn_vs_um(mt_path)

  0%|          | 0/21 [00:00<?, ?it/s]